# **Generative LLM**

* Notebook to generate speeches of german politicians
* Model used: german version of Llama 2, with 13 billion parameters $\to$ "jphme/Llama-2-13b-chat-german" [link to model](https://huggingface.co/jphme/Llama-2-13b-chat-german)
* Furthermore, tested model answers on political survey data ("Wahl-O-Mat")

In [ ]:
# install required packages

!pip install datasets
!pip install torch
!pip install transformers
!pip install peft
!pip install bitsandbytes -U
!pip install -U trl
!pip install nltk
!pip install faiss-cpu
!pip install sentence_transformers

  Using cached peft-0.17.0-py3-none-any.whl.metadata (14 kB)
Using cached peft-0.17.0-py3-none-any.whl (503 kB)
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl (72.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 11.7 MB/s eta 0:00:0000:01
  Using cached faiss_cpu-1.11.0.post1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.0 kB)
  Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached faiss_cpu-1.11.0.post1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (31.3 MB)
Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into acc

In [ ]:
# import packages and settings

import subprocess
import sys
from datasets import Dataset
import re
import string
import seaborn as sns
import os
import pandas as pd
import random
import torch 
import numpy as np
from datasets import Dataset
import bitsandbytes as bnb
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    GenerationConfig
)
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
import warnings
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
import faiss
import json
from tqdm import tqdm

warnings.filterwarnings('ignore')
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.empty_cache()

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## **1) Import and Clean Speech Data**

* Clean linebreaks
* Limit speech length (to 200 - 1500 words)
* Remove greetings
* Sample to have same number of speeches per party $\to$ prevent potential knowledge bias

In [ ]:
data = pd.read_csv("../data/final_data.csv", on_bad_lines='skip')

if "Unnamed: 0" in data.columns:
    data = data.drop("Unnamed: 0", axis=1)

data["party"] = data["party"].replace("CDU/CSU", "Union")
data

,speech_text,legislative_period,protocol_nr,agenda_item_number,party,agenda_item_title,date,full_name
0,Herr Präsident! Kolleginnen und Kollegen! Die ...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Hans Michelbach
1,Sehr geehrter Herr Präsident! Liebe Kolleginne...,19,10,3,SPD,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Ingrid Arndt-Brauer
2,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Antje Tillmann
3,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,GRÜNE,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Gerhard Schick
4,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,19,10,3,FDP,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Florian Toncar
...,...,...,...,...,...,...,...,...
36112,Sehr geehrter Herr Präsident! Liebe Kolleginne...,20,99,5,SPD,Wolfsbestandsmanagement,2023-04-26,Lina Seitzl
36113,Frau Präsidentin! Meine Damen und Herren! Sie ...,20,99,3,GRÜNE,Aktuelle Stunde - Umstrittene Personalpolitik ...,2023-04-26,Till Steffen
36114,Sehr geehrter Herr Präsident! Meine Damen und ...,20,99,4,AfD,Bundeswehreinsatz Evakuierung aus Sudan,2023-04-26,Joachim Wundrak
36115,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,20,99,7,LINKE,Unregulierte Massenmigration,2023-04-26,Clara Bünger


In [10]:
# clean line breaks and special spaces
data["cleaned_text"] = data["speech_text"].apply(lambda x: x.replace("\xa0", " "))
data["cleaned_text"] = data["cleaned_text"].apply(lambda x: x.replace("\n", " "))

# remove repeated spaces
data["cleaned_text"] = data["cleaned_text"].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

In [11]:
# extract count of words
data["word_count"] = data["cleaned_text"].apply(lambda x: len(x.split()))

# limit speech length
data_filtered = data[data["word_count"] >= 200][data["word_count"] < 1501]

print("Nr. of Training Speeches:", len(data_filtered))
data_filtered

Nr. of Training Speeches: 36117


,speech_text,legislative_period,protocol_nr,agenda_item_number,party,agenda_item_title,date,full_name,cleaned_text,word_count
0,Herr Präsident! Kolleginnen und Kollegen! Die ...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Hans Michelbach,Herr Präsident! Kolleginnen und Kollegen! Die ...,520
1,Sehr geehrter Herr Präsident! Liebe Kolleginne...,19,10,3,SPD,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Ingrid Arndt-Brauer,Sehr geehrter Herr Präsident! Liebe Kolleginne...,693
2,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,Union,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Antje Tillmann,Herr Präsident! Liebe Kolleginnen und Kollegen...,710
3,Herr Präsident! Liebe Kolleginnen und Kollegen...,19,10,3,GRÜNE,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Gerhard Schick,Herr Präsident! Liebe Kolleginnen und Kollegen...,752
4,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,19,10,3,FDP,Aktuelle Stunde zu einer europäischen Bankenunion,2018-01-31,Florian Toncar,Liebe Kolleginnen und Kollegen! Es ist vorhin ...,872
...,...,...,...,...,...,...,...,...,...,...
36112,Sehr geehrter Herr Präsident! Liebe Kolleginne...,20,99,5,SPD,Wolfsbestandsmanagement,2023-04-26,Lina Seitzl,Sehr geehrter Herr Präsident! Liebe Kolleginne...,837
36113,Frau Präsidentin! Meine Damen und Herren! Sie ...,20,99,3,GRÜNE,Aktuelle Stunde - Umstrittene Personalpolitik ...,2023-04-26,Till Steffen,Frau Präsidentin! Meine Damen und Herren! Sie ...,789
36114,Sehr geehrter Herr Präsident! Meine Damen und ...,20,99,4,AfD,Bundeswehreinsatz Evakuierung aus Sudan,2023-04-26,Joachim Wundrak,Sehr geehrter Herr Präsident! Meine Damen und ...,611
36115,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,20,99,7,LINKE,Unregulierte Massenmigration,2023-04-26,Clara Bünger,Sehr geehrte Präsidentin! Liebe Kolleginnen un...,480


#### **Remove beginning/greeting of Speeches**

* Initial trys suggested that the model might get too caught in lerning to reproduce these starts, instead of the speeches content

In [16]:
# list of greeting-related words
greeting_words = [
    "Damen", "Herren", "Herr", "Kollegen", "Kolleginnen", "Präsident", "Präsidentin",
    "Kollege", "Kollegin", "verehrte", "verehrten", "geehrte", "geehrter", "geehrten"
]

# Build a regex pattern to match any of these words as whole words (case insensitive)
greeting_pattern = re.compile(r'\b(?:' + '|'.join(greeting_words) + r')\b', flags=re.IGNORECASE)

def remove_greeting_sentences(text, max_sentences=10):
    # Tokenize into sentences
    sentences = sent_tokenize(text, language='german')
    
    cleaned_sentences = []
    for i, sentence in enumerate(sentences):
        if i < max_sentences and greeting_pattern.search(sentence):
            continue  # Skip greeting sentence
        cleaned_sentences.append(sentence)
    
    return ' '.join(cleaned_sentences)

# apply
data_filtered['cleaned_text'] = data_filtered['cleaned_text'].apply(remove_greeting_sentences)

# sainity check
print(data_filtered["cleaned_text"][100])

Als ich 2013 ganz frisch im Bundestag die Berichterstattung für digitale Bildung übernommen habe, wusste mit diesen Begriffen noch kaum jemand etwas anzufangen, außer natürlich einer eingeschworenen Gemeinschaft digitalaffiner Lehrkräfte im Twitterlehrerzimmer. Seither sind wir wesentlich weitergekommen. Das freut mich ungemein. Dann kam die Ankündigung der damaligen Bildungsministerin Wanka eines DigitalPakts in der „BamS“. Jetzt haben wir den DigitalPakt, und zwar mit Brief und Siegel. Das ist großartig. Das ist ein Anlass zur Freude, zum Durchatmen, aber bitte nicht zu lang. Wir leben in einer Welt, die niemals stehen bleibt, in der digitale Medien, die Methode digitalen Lernens und Arbeitens immer selbstverständlicher werden. Die Schule muss dazu ermutigen und befähigen. Insofern bin ich der Opposition dankbar, als sie den DigitalPakt weiterdenkt und weiterentwickelt. Das steht heute nicht an, aber es muss weitergehen, ganz klar. Auch ich bin überzeugt: Der DigitalPakt muss dauerha

#### **Import Filteres Party Manifesto Parts on Topics**

* We selected four topic, the models shall generate speeches to
* Manifesto texts were searches for the keywords (and synonymes), and the reffering sentences as well as 3 sentences before and after the target sentence were extracted

In [47]:
with open("../data/party_manifesto_contexts.json", "r", encoding="utf-8") as f:
    party_manifesto_contexts = json.load(f)

#party_manifesto_contexts

#### **Sample Speeches to Same Sount**

* Ensure same number of training speeches per party

In [17]:
parties_count = data_filtered[["cleaned_text", "party"]].groupby("party").count().reset_index()
parties_count

,party,cleaned_text
0,AfD,5041
1,FDP,4683
2,GRÜNE,5067
3,LINKE,3618
4,SPD,8032
5,Union,9676


In [ ]:
# for a balanced training sample, restrict all parties to the minimum of speeches (LINKE)

max_length = parties_count["cleaned_text"].min() #3618
parties = data_filtered["party"].unique()


# construct final df
final_df = pd.DataFrame()


for party in parties:

    party_unique_df = data_filtered[data_filtered["party"] == party]

    if len(party_unique_df) >= max_length: # if too many speeches, sample
        party_sample = party_unique_df.sample(n=max_length, random_state=42)
    else:
        party_sample = party_unique_df


    final_df = pd.concat([final_df, party_sample], ignore_index=True)


final_df.head()

,speech_text,legislative_period,protocol_nr,agenda_item_number,party,agenda_item_title,date,full_name,cleaned_text,word_count
0,Sehr geehrte Frau Präsidentin! Werte Kolleginn...,20,79,12,Union,EU-Richtlinie Umweltauswirkungen Kunststoffpro...,2023-01-19,Björn Simon,Auch von meiner Seite aus einen herzlichen Glü...,789
1,Hochgeschätzter Herr Präsident! Kolleginnen un...,19,23,6,Union,Verkehr und digitale Infrastruktur,2018-03-22,Andreas Scheuer,Luftqualität ist Lebensqualität; aber Lebensqu...,1292
2,Frau Präsidentin! Verehrte Kolleginnen und Kol...,20,18,8,Union,Aktuelle Stunde - Bundeswehreinsätze in Mali b...,2022-02-18,Reinhard Brandl,Die Ampelkoalition kann jetzt nichts für die S...,741
3,Frau Präsidentin! Liebe Kolleginnen und Kolleg...,20,21,4,Union,Meinungsfreiheit in Sozialen Medien,2022-03-17,Marc Henrichmann,Das vierte Wort im Antrag der AfD ist „Russlan...,936
4,Herr Präsident! Meine sehr verehrten Damen und...,19,164,3,Union,Bundeswehreinsatz EUTM Mali,2020-05-29,Johann David Wadephul,Die Lage im Sahel ist kritisch. Sie ist sogar ...,1011


## **2.) Import the Base Model**

* 

In [ ]:
# Model configuration
model_name = "jphme/Llama-2-13b-chat-german"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Loading Llama model: {model_name}")
print(f"Device: {device}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Enable Quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.bfloat16 
)


# Load LLama model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    quantization_config = quantization_config,
    device_map="auto" if torch.cuda.is_available() else None,
)

print("Llama model and tokenizer loaded successfully!")
print(f"Model parameters: {base_model.num_parameters():,}")


Loading Llama model: jphme/Llama-2-13b-chat-german
Device: cuda


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Llama model and tokenizer loaded successfully!
Model parameters: 13,015,864,320


In [ ]:
# create the chat template and construct the system prompt
# different instructions were tested, i.e. initial speeches often contained englisch sentences. 
# added an explicit instruction to write german.

tokenizer.chat_template = """<s>[INST] <<SYS>>Du bist ein hilfreicher, respektvoller und ehrlicher Assistent.
Du bist ein*e Abgeordnete*r im deutschen Bundestag.
Gegeben einer Parteizugehörigkeit und eines Themas oder Titels, schreibst du eine politische Rede.
Du darfst nur auf Deutsch schreiben.
Der Umfang sind 200 bis 1500 Worte.
<</SYS>>

{% for message in messages %}
{{ message['content'] }}{% if not loop.last %} [/INST] <s>[INST] {% else %} [/INST] {% endif %}
{% endfor %}</s>"""

tokenizer.model_max_length = 2048

## **3.) Initial tries for baseline speeches, produced given a title**

* To get a first idea about the model performance, we decided to let the model generate speeches for a given party and speech title
* Since the training data contain speech titles, we were able to compare the resulting generated and true speeches.

In [ ]:
def generate_prediction_base(fraction_label, title, model, tokenizer):
    
    """Generate politicat party speeches for a given title using the base Llama model with chat template"""
    
    # user prompt
    messages = [
        {
            "role": "user",
            "content": f" Du bist Abgeordnete*r im deutschen Bundestag. Deine Fraktion ist: {fraction_label}. Schreibe eine deutsche, politische Rede mit bis zu 1500 Worten, die du im Bundestag hälst. Der Titel ist {title}."
        }
    ]

    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False 
    )

    model_inputs = tokenizer([formatted_text], return_tensors="pt", max_length=2048, truncation=True).to(model.device)

    # generation
    with torch.inference_mode():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048,
            temperature=0.1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,  # Ensure EOS token stops generation
        )

    # Extract and decode
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return content
    

In [ ]:
# sample an actual speech title per party

random_title_info = {}

for party in final_df["party"].unique():
    sample = final_df[final_df["party"] == party].sample(n=1, random_state=23)
    index = sample.index[0]
    title = sample["agenda_item_title"].values[0]
    random_title_info[party] = {"index": index, "title": title}

random_title_info

{'Union': {'index': 713, 'title': 'Regulierung von Bonitätsauskünften'},
 'SPD': {'index': 4331,
  'title': 'Bildung und Forschung für geflüchtete Ukrainer'},
 'GRÜNE': {'index': 7949,
  'title': 'Verteidigungspolitik, Einsatzbereitschaft Bundeswehr'},
 'FDP': {'index': 11567,
  'title': 'Aktuelle Stunde zur Eskalation in der Golfregion'},
 'AfD': {'index': 15185, 'title': 'Kinderzukunftsprogramm'},
 'LINKE': {'index': 18803, 'title': 'Netzwerkdurchsetzungsgesetz'}}

#### **Generate Speeches per Party and Title**

In [ ]:
# make predictions for base model on title

prediction_base = []

for party, value in random_title_info.items():
    print(party)
    prediction_base.append(generate_prediction_base(party, value["title"], base_model, tokenizer))

Union
SPD
GRÜNE
FDP
AfD
LINKE


In [27]:
prediction_base_df = pd.DataFrame(prediction_base).rename(columns = {0 : "speech"})
prediction_base_df["party"] =  list(random_title_info.keys())
prediction_base_df.to_csv("../data/generative_predictions_base_V2.csv")
prediction_base_df

,speech,party
0,#Regulierung von Bonitätsauskünften#\n\nHerr P...,Union
1,#Hochschulzugang für geflüchtete Ukrainer#\n\n...,SPD
2,"#Verteidigungspolitik#\n\nHerr Präsident, vere...",GRÜNE
3,"#Halte 1#\n\nHerr Präsident, verehrtes Haus,\n...",FDP
4,"#Hallo, meine Damen und Herren,\n\nheute möcht...",AfD
5,#Netzwerkdurchsetzungsgesetz#\n\nHerr Präsiden...,LINKE


In [ ]:
# comparison 

for i, (party, value) in enumerate(random_title_info.items()):
    
    true_speech = final_df.loc[final_df['agenda_item_title'] == value["title"], 'speech_text'].values[0]
    predicted_speech = prediction_base[i]

    # Print comparison
    print("\n===================")
    print(f"=== {party} ===")
    print(f"Agenda Item: {value['title']}")
    print("===================")

    print(" True Speech:")
    print("===================")
    print(true_speech)
    
    print("Predicted Speech:")
    print("===================")
    print(predicted_speech)


=== Union ===
Agenda Item: Regulierung von Bonitätsauskünften
 True Speech:
Ich warte gern noch etwas, wenn Sie mich weiter loben wollen, Frau Präsidentin.
Sonntag ist der erste Advent.
Sehr geehrte Frau Präsidentin! Liebe Kolleginnen! Liebe Kollegen! In unserem Land besteht grundsätzlich die im Grundgesetz verankerte Vertragsfreiheit. Mit der hat eine Partei in diesem Haus immer wieder sichtliche Probleme. Lieber Matthias Birkwald, liebe Frau Nastic, heute Nachmittag haben wir hier über die Frage eines Kündigungsschutzes für über 70-jährige Mieter debattiert.
Das klingt toll, das klingt gut, aber das ist bei Ihren Anträgen immer so. Es gibt da immer zwei Seiten der Medaille, eine positive, die verlockend klingt – das ist der Kündigungsschutz für über 70-jährige Mieter –, aber gleichzeitig die Rückseite der Medaille, nämlich die Problematik, dass dann ein 65-jähriger, 67-jähriger, 68-jähriger Mietinteressent gar keinen Mietvertrag mehr bekommen wird.
Das heißt ja im Endeffekt: Was imm

**Interpretation**

* Results often show the same pattern: Greeting, then "today I want to discuss a topic, important to us as a whole society: {topic}. We as {party} ..."
* Repetitions of whole paragraphs, i.e. for FDP
* Speeches are all german

## **4.) Fine Tuning with True Political Speeches**

* re-import base model for fine-tuning
* format dataset for training
* train and save

In [ ]:
# reload new model --> use this for fine tuning 

model_name = "jphme/Llama-2-13b-chat-german"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading Llama model: {model_name}")
print(f"Device: {device}")


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.bfloat16 
)


# Load LLama model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    quantization_config = quantization_config,
    device_map="auto" if torch.cuda.is_available() else None,
)

print("Llama model and tokenizer loaded successfully!")
print(f"Model parameters: {model.num_parameters():,}")

Loading Llama model: jphme/Llama-2-13b-chat-german
Device: cuda


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Llama model and tokenizer loaded successfully!
Model parameters: 13,015,864,320


In [ ]:
# add the chat template with system prompt

tokenizer.chat_template = """<s>[INST] <<SYS>>
Du bist ein hilfreicher, respektvoller und sachlicher Assistent. 
Du bist Abgeordnete*r im Deutschen Bundestag.
Gegeben sind eine Parteizugehörigkeit und ein Tagesordnungspunkt. 
Deine Aufgabe ist es, eine überzeugende politische Rede auf DEUTSCH zu verfassen.

Rahmenbedingungen:
- Sprache: ausschließlich Deutsch
- Stil: parlamentarisch, sachlich, aber engagiert
- Inhalt: Position und Werte deiner Partei deutlich machen
- Länge: zwischen 200 und 1500 Wörtern

<</SYS>>

{% for message in messages %}
{{ message['content'] }}{% if not loop.last %} [/INST] <s>[INST] {% else %} [/INST] {% endif %}
{% endfor %}</s>"""

tokenizer.model_max_length = 4096 # increased

#### **Format the Prompt, create the Dataset**

In [ ]:
# function to format prompts
# user prompt got adapted, to be more explicit

def row_to_prompt(row):
    messages = [
        {
            "role": "user",
            "content": f"Du bist Abgeordnete*r im Deutschen Bundestag. Deine Fraktion ist: {row['party']}. Schreibe eine politische Rede auf DEUTSCH. Der Titel ist: {row['agenda_item_title']}."
        },
        {
            "role": "assistant",
            "content": row['cleaned_text']
        }
    ]

    # apply chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    return prompt


# make dataset 
final_df["prompt"] = final_df.apply(row_to_prompt, axis=1)
dataset = Dataset.from_pandas(final_df[["prompt"]])

In [ ]:
# tokenize the resulting chat formated prompt

def tokenize(example):
    result = tokenizer(
        example["prompt"],
        padding="max_length",
        truncation=True,
        max_length=2048,
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["prompt"])

Map:   0%|          | 0/21708 [00:00<?, ? examples/s]

In [39]:
# training settings

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./llama2-all-speeches_V2",
    per_device_train_batch_size=1, 
    gradient_accumulation_steps=8, 
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)



In [ ]:
trainer = SFTTrainer(
    model=model, 
    train_dataset=tokenized_dataset, 
    args=training_args,
    peft_config=lora_config
)

# save the trainer with learned weights
trainer.train()
trainer.save_model("./Llama_all_speeches_V2")

Truncating train dataset:   0%|          | 0/21708 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,14.610800
20,5.713300
30,2.979300
40,2.686700
50,1.860800
60,1.127400
70,0.721300
80,0.593200
90,0.582800
100,0.544700


In [ ]:
# this save did not work as intended: apparently, this only saved the base model, not including the fine-tuned parameters
# later: reimport trained parameters over trainer and PEFT modules

# gen_config = GenerationConfig(
#     do_sample=False,
#     temperature=None,
#     top_p=None
# )

# model.generation_config = gen_config

# model.save_pretrained("generator_ft_V2/")
# tokenizer.save_pretrained("generator_ft_V2/")

('generator_ft_V2/tokenizer_config.json',
 'generator_ft_V2/special_tokens_map.json',
 'generator_ft_V2/chat_template.jinja',
 'generator_ft_V2/tokenizer.json')

#### **Title Speech Pediction**

* For comparison, we also generated speeches using the title and the new, fine-tuned model 
* New function, to incliude temperature as a parameter 

In [ ]:

def generate_prediction_after_ft(fraction_label, title, model, tokenizer, temp):
    """Generate politicat party speeches using the base Llama model with chat template"""
    
    messages = [
        {
            "role": "user",
            "content": f" Du bist Abgeordnete*r im deutschen Bundestag. Deine Fraktion ist: {fraction_label}. Schreibe eine politische Rede auf DEUTSCH. Der Titel ist {title}."
        },
    ]


    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False 
    )

    model_inputs = tokenizer([formatted_text], return_tensors="pt", max_length=2048, truncation=True).to(model.device)


    with torch.inference_mode():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048,
            temperature=temp, # included
            repetition_penalty = 1.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,           
        )

    # Extract and decode
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return content

In [39]:
# temperature at 0.1

prediction_after_finetune_01 = []

for party, value in random_title_info.items():
    print(party)
    print(value["title"])
    prediction_after_finetune_01.append(generate_prediction_after_ft(party, value["title"], model, tokenizer, temp = 0.1))

Union
Regulierung von Bonitätsauskünften
SPD
Bildung und Forschung für geflüchtete Ukrainer
GRÜNE
Verteidigungspolitik, Einsatzbereitschaft Bundeswehr
FDP
Aktuelle Stunde zur Eskalation in der Golfregion
AfD
Kinderzukunftsprogramm
LINKE
Netzwerkdurchsetzungsgesetz


In [40]:
# temperature at 0.3

prediction_after_finetune_03 = []

for party, value in random_title_info.items():
    print(party)
    print(value["title"])
    prediction_after_finetune_03.append(generate_prediction_after_ft(party, value["title"], model, tokenizer, temp = 0.3))

Union
Regulierung von Bonitätsauskünften
SPD
Bildung und Forschung für geflüchtete Ukrainer
GRÜNE
Verteidigungspolitik, Einsatzbereitschaft Bundeswehr
FDP
Aktuelle Stunde zur Eskalation in der Golfregion
AfD
Kinderzukunftsprogramm
LINKE
Netzwerkdurchsetzungsgesetz


In [ ]:
# export individual

prediction_after_finetune_01_df = pd.DataFrame(prediction_after_finetune_01).rename(columns = {0 : "speech"})
prediction_after_finetune_01_df["party"] =  list(random_title_info.keys())
prediction_after_finetune_01_df.to_csv("../data/generative_prediction_after_finetune_01.csv")

prediction_after_finetune_03_df = pd.DataFrame(prediction_after_finetune_03).rename(columns = {0 : "speech"})
prediction_after_finetune_03_df["party"] =  list(random_title_info.keys())
prediction_after_finetune_03_df.to_csv("../data/generative_prediction_after_finetune_03.csv")
prediction_after_finetune_03_df

,speech,party
0,# INFOS SEITE DU BIST EIN HILFEZWEIG FÜR SICHE...,Union
1,[INST] Sehr geehrte Frau Präsidentin! Liebe Ko...,SPD
2,#INCLUDE <https://www.youtube-nocookie.com/_Yl...,GRÜNE
3,#NeverAgain – so lautet das Motto des jüngsten...,FDP
4,[_] Sehr geehrte Frau Präsidentin! Verehrte Ko...,AfD
5,[INST] Sehr geehrte Frau Präsidentin! Liebe Ko...,LINKE


In [ ]:
# optional reimport data
#prediction_base_df = pd.read_csv("../data/generative_predictions_base.csv").drop(columns = "Unnamed: 0")
#prediction_after_finetune_01_df = pd.read_csv("../data/generative_prediction_after_finetune_01.csv").drop(columns = "Unnamed: 0")
#prediction_after_finetune_03_df = pd.read_csv("../data/generative_prediction_after_finetune_03.csv").drop(columns = "Unnamed: 0")

# rename speeches
prediction_base_df = prediction_base_df.rename(columns = {"speech" : "speech_base"})
prediction_after_finetune_01_df = prediction_after_finetune_01_df.rename(columns = {"speech" : "speech_ft_01"})
prediction_after_finetune_03_df = prediction_after_finetune_03_df.rename(columns = {"speech" : "speech_ft_03"})
prediction_after_finetune_03_df

# merge to one df
speeches_df = (
    prediction_base_df
    .merge(prediction_after_finetune_01_df, on="party")
    .merge(prediction_after_finetune_03_df, on="party")
)

# export
speeches_df.to_csv("../data/test_speeches_by_title.csv", index = False)

In [ ]:
# compare performance

for i, (party, value) in enumerate(random_title_info.items()):
    
    # true speech
    true_speech = final_df.loc[final_df['agenda_item_title'] == value["title"], 'speech_text'].values[0]

    # Get the predicted speech
    predicted_speech = prediction_base[i]
    finetuned_speech_01 = prediction_after_finetune_01[i]
    finetuned_speech_03 = prediction_after_finetune_03[i]

    # Print comparison
    print("==================")
    print(f"\n=== {party} ===")
    print(f"Agenda Item: {value['title']}\n")
    print("==================")
    
    print("True Speech:\n")
    print("==================")
    print(true_speech)
    
    print("Initial predicted Speech:\n")
    print("==================")
    print(predicted_speech)
    
    print("Fine-tuned predicted (0.1 temp) Speech:\n")
    print("==================")
    print(finetuned_speech_01)
    
    print("Fine-tuned predicted (0.3 temp) Speech:\n")
    print("==================")
    print(finetuned_speech_03)


=== Union ===
Agenda Item: Regulierung von Bonitätsauskünften

True Speech:

Ich warte gern noch etwas, wenn Sie mich weiter loben wollen, Frau Präsidentin.
Sonntag ist der erste Advent.
Sehr geehrte Frau Präsidentin! Liebe Kolleginnen! Liebe Kollegen! In unserem Land besteht grundsätzlich die im Grundgesetz verankerte Vertragsfreiheit. Mit der hat eine Partei in diesem Haus immer wieder sichtliche Probleme. Lieber Matthias Birkwald, liebe Frau Nastic, heute Nachmittag haben wir hier über die Frage eines Kündigungsschutzes für über 70-jährige Mieter debattiert.
Das klingt toll, das klingt gut, aber das ist bei Ihren Anträgen immer so. Es gibt da immer zwei Seiten der Medaille, eine positive, die verlockend klingt – das ist der Kündigungsschutz für über 70-jährige Mieter –, aber gleichzeitig die Rückseite der Medaille, nämlich die Problematik, dass dann ein 65-jähriger, 67-jähriger, 68-jähriger Mietinteressent gar keinen Mietvertrag mehr bekommen wird.
Das heißt ja im Endeffekt: Was im

**Interpretation:**

* unofrtunately, even though selecting a german model, the fine-tuned speeches have grammar issues.
* furthermore, the model does not seem to adjust the speech to the given title, but discusses arbitrary tooics (probably what it has seen during training).The base model was better at talking about the desired topic


## **5.) Generate Speeches on desired Topics**

* Reimport model, add LoRA weights

#### **Reimport base model, add trained weights and tokenizer**

In [ ]:
# reimport, first base model, add LoRA weights on top 

model_name = "jphme/Llama-2-13b-chat-german"
device = "cuda" if torch.cuda.is_available() else "cpu"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Base Model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config
)

# Load LoRA adapter on top
model = PeftModel.from_pretrained(base_model, "./Llama_all_speeches_V2")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# add chat template with system prompt

tokenizer.chat_template = """<s>[INST] <<SYS>>
Du bist ein hilfreicher, respektvoller und sachlicher Assistent. 
Du bist Abgeordnete*r im Deutschen Bundestag.
Gegeben sind eine Parteizugehörigkeit und ein Tagesordnungspunkt. 
Deine Aufgabe ist es, eine überzeugende politische Rede auf DEUTSCH zu verfassen.

Rahmenbedingungen:
- Sprache: ausschließlich Deutsch
- Stil: parlamentarisch, sachlich, aber engagiert
- Inhalt: Position und Werte deiner Partei deutlich machen
- Länge: zwischen 200 und 1500 Wörtern

<</SYS>>

{% for message in messages %}
{{ message['content'] }}{% if not loop.last %} [/INST] <s>[INST] {% else %} [/INST] {% endif %}
{% endfor %}</s>"""

tokenizer.model_max_length = 4096

In [33]:
# Merge LoRA adapters into the base model
merged_model = model.merge_and_unload()

# Define a new, valid generation config for deterministic generation
gen_config = GenerationConfig(
    do_sample=False  # sampling disabled, all other sampling params ignored
)

# Assign this new config before saving
merged_model.generation_config = gen_config

# Save merged model, tokenizer, and clean generation config
merged_model.save_pretrained("generator_final_V2/")
tokenizer.save_pretrained("generator_final_V2/")
gen_config.save_pretrained("generator_final_V2/")  

#### **Generate Speeches**

In [ ]:
# generate speeches for desired topic 


def generate_speech_to_topic(fraction_label, topic, model, tokenizer, temp):
    """Generate politicat party speeches on specific topics using the base Llama model with chat template"""
    
    # adpated prompt
    messages = [
        {
            "role": "user",
            "content": f" Du bist Abgeordnete*r im deutschen Bundestag. Deine Fraktion ist: {fraction_label}. Schreibe eine politische Rede auf DEUTSCH. Es soll um das Thema {topic} gehen."
        },
    ]

    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  
    )
    model_inputs = tokenizer([formatted_text], return_tensors="pt", max_length=2048, truncation=True).to(model.device)

    with torch.inference_mode():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048,
            temperature=temp,
            repetition_penalty = 1.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,         
        )

    # Extract and decode 
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return content

#### **Set Topics for Speeches**
$\to$ manually decided topics

In [47]:
# dictionary of party and topics

topic_to_speech = {"Union" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "SPD" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "GRÜNE" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "FDP" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "AfD" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"],
                   "Linke" : ["Mindestlohn", "Bundeswehreinsatz im Kosovo", "Wirtschaftshilfen Corona", "Gaspreise"]}

#### **Base Model and Fine-Tuned Model on Topics as Comparison**

* Tried two temperatures: 0.1 and 0.3 for comparison

In [ ]:
# apply generate_speech_to_topic to base_model and model (ft) --> Temperature = 0.1

base_model_topic_01 = {}
ft_model_topic_01 = {}

for party, topic in topic_to_speech.items():
    print(party)
    base_model_topic_01[party] = []
    ft_model_topic_01[party] = []
    
    for top in topic:
        base_model_topic_01[party].append(generate_speech_to_topic(party, top, base_model, tokenizer, temp = 0.1)) # generate
        ft_model_topic_01[party].append(generate_speech_to_topic(party, top, model, tokenizer, temp = 0.1)) # generate


# make a DataFrame
data_rows = []
for party in base_model_topic_01:
    topics = topic_to_speech[party]
    base_speeches = base_model_topic_01[party]
    ft_speeches = ft_model_topic_01[party]
    
    for topic, base_speech, ft_speech in zip(topics, base_speeches, ft_speeches):
        data_rows.append({
            "party": party,
            "topic": topic,
            "speech_base_model": base_speech,
            "speech_ft_model": ft_speech
        })

speeches_topic_df_01 = pd.DataFrame(data_rows)
speeches_topic_df_01.to_csv("../data/speeches_by_topic_01_V2.csv", index = False)
speeches_topic_df_01

Union
SPD
GRÜNE
FDP
AfD
Linke


,party,topic,speech_base_model,speech_ft_model
0,Union,Mindestlohn,#HartzIV - Die Zeit der Verantwortung für die ...,[INST] Herr Präsident! Liebe Kolleginnen und l...
1,Union,Bundeswehreinsatz im Kosovo,"import_contacts() { echo ""Hallo zusammen! Ich ...",[INST] Herr Präsident! Liebe Kolleginnen und l...
2,Union,Wirtschaftshilfen Corona,"#Herr Präsident! Mein Name bin(d) (Name), ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...
3,Union,Gaspreise,#Hochfrequenzübertragung#\n\nSehr verehrte Dam...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...
4,SPD,Mindestlohn,#HartzIV - Ein neuer Start für die soziale Sic...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...
5,SPD,Bundeswehreinsatz im Kosovo,"#Herr Präsident! Mein Name bin(n) (Name), ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...
6,SPD,Wirtschaftshilfen Corona,"#Herr Präsident! Mein Name bin(n) (Name), ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...
7,SPD,Gaspreise,#Hallo! Ich bin die Sprecherin der sozialdemok...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...
8,GRÜNE,Mindestlohn,#Mindesteinkommen für alle!# Ein neuer Start i...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...
9,GRÜNE,Bundeswehreinsatz im Kosovo,"import_contacts() { echo ""Hallo zusammen! Ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...


In [ ]:
# apply generate_speech_to_topic to base_model and model (ft) --> Temperature = 0.3

base_model_topic = {}
ft_model_topic = {}

for party, topic in topic_to_speech.items():
    print(party)
    base_model_topic[party] = []
    ft_model_topic[party] = []
    
    for top in topic:
        base_model_topic[party].append(generate_speech_to_topic(party, top, base_model, tokenizer, temp = 0.3))
        ft_model_topic[party].append(generate_speech_to_topic(party, top, model, tokenizer, temp = 0.3))


data_rows = []

for party in base_model_topic:
    topics = topic_to_speech[party]
    base_speeches = base_model_topic[party]
    ft_speeches = ft_model_topic[party]
    
    for topic, base_speech, ft_speech in zip(topics, base_speeches, ft_speeches):
        data_rows.append({
            "party": party,
            "topic": topic,
            "speech_base_model": base_speech,
            "speech_ft_model": ft_speech
        })

speeches_topic_df_03 = pd.DataFrame(data_rows)
speeches_topic_df_03.to_csv("../data/speeches_by_topic_03_V2.csv", index = False)
speeches_topic_df_03

Union
SPD
GRÜNE
FDP
AfD
Linke


,party,topic,speech_base_model,speech_ft_model
0,Union,Mindestlohn,#HartzIV - Die Zeit der Verantwortung für die ...,[Inst] Frau Präsidentin! Liebe Kolleginnen und...
1,Union,Bundeswehreinsatz im Kosovo,import('./dein_vorschlag') {\n```scss\nHerr Pr...,[INST] Herr Präsident! Liebe Kolleginnen und l...
2,Union,Wirtschaftshilfen Corona,#Hallo! Ich bin die Kanzlermeinung der CDU / C...,[Inst] Frau Präsidentin! Liebe Kolleginnen und...
3,Union,Gaspreise,"import_contacts() {print(""Hallo zusammen!"")} ...",[INST] Frau Präsidentin! Liebe Kolleginnen und...
4,SPD,Mindestlohn,import('./dein_abstimmungsverhalten'); // Erse...,[INST] Sehr verehrte Frau Präsidentin! Meine s...
5,SPD,Bundeswehreinsatz im Kosovo,import('./dein_text') // Ersetze dies durch de...,[INST] Sehr verehrte Frau Präsidentin! Meine s...
6,SPD,Wirtschaftshilfen Corona,#Hallo! Ich bin die Kanzlermeinung der soziald...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...
7,SPD,Gaspreise,"import(""https://de.wikipedia.org"")(function($)...",[INST] Sehr verehrte Frau Präsidentin! Meine s...
8,GRÜNE,Mindestlohn,#Mindesteinkommen für alle!# Ein neuer Start i...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...
9,GRÜNE,Bundeswehreinsatz im Kosovo,"import_contacts() { echo ""import contact"" }",[INST] Frau Präsidentin! Liebe Kolleginnen und...


In [ ]:
# compare 
speeches_topic_df_01 = pd.read_csv("../data/speeches_by_topic_01_V2.csv")
speeches_topic_df_03 = pd.read_csv("../data/speeches_by_topic_03_V2.csv")

# Union, Bundeswehreinsatz 
print(speeches_topic_df_01["speech_ft_model"][1])
print(speeches_topic_df_03["speech_ft_model"][1])

[INST] Herr Präsident! Liebe Kolleginnen und lieber Kollegen! Mein Vorredner hat schon darauf verwiesen – ich möchte noch mal daran erinnern; denn die Debatte zeigt ja auch wieder einmal den Unterschied in der Sichtweise zur Außenpolitik dieser Koalition gegenüber dem Blickwinkel von CDU und CSU – dass wir als Unionsfraktionen immer sehr genau hinschauen werden bei jeder Entscheidung für einen militärischen Einsatz unseres Landes mit Soldatinnen und Soldaten an einem anderen Ort dieses Planeten. Wir haben unsere Verantwortung vor Augen geführt beim Beschluss zum KFOR-Einsatz hier am letzten Donnerstagabend nach langem Nachdenken gemeinsam getroffen. Ich will Ihnen ganz klar versichert geben: Auch wenn man sich nicht mehr so richtig entscheiden kann wie früher wegen des Ausstiegs aus Afghanistan oder weil Sie eben gesagt hatten, was da passieren könnte, wer wann wo angreifen wird usw., dann bleibt doch festzuhalten, meine Damen und Herren: Die Lage dort war nie weniger bedenkbar gewesen

**Interpretation**

* Sentence structure contains grammar mistakes
* Model invents new, non-true words like: Parlamentsvereinbardei, Ansinnigpakt --> model with 0.3 temperature
* Erstly, Zweitly : model merges german and englisch

In [ ]:
# combine both dfs to one
speeches_topic_df_01 = speeches_topic_df_01.rename(columns = {"speech_base_model" : "speech_base_model_01", "speech_ft_model" : "speech_ft_model_01"})
speeches_topic_df_03 = speeches_topic_df_03.rename(columns = {"speech_base_model" : "speech_base_model_03", "speech_ft_model" : "speech_ft_model_03"})

speeches_finetuned_V2 = (
    speeches_topic_df_01
    .merge(speeches_topic_df_03, on=["party", "topic"], how = "inner")
)
speeches_finetuned_V2.to_csv("../data/generated_speeches_final_V2.csv", index = False)
speeches_finetuned_V2

,party,topic,speech_base_model_01,speech_ft_model_01,speech_base_model_03,speech_ft_model_03
0,Union,Mindestlohn,#HartzIV - Die Zeit der Verantwortung für die ...,[INST] Herr Präsident! Liebe Kolleginnen und l...,#HartzIV - Die Zeit der Verantwortung für die ...,[Inst] Frau Präsidentin! Liebe Kolleginnen und...
1,Union,Bundeswehreinsatz im Kosovo,"import_contacts() { echo ""Hallo zusammen! Ich ...",[INST] Herr Präsident! Liebe Kolleginnen und l...,import('./dein_vorschlag') {\n```scss\nHerr Pr...,[INST] Herr Präsident! Liebe Kolleginnen und l...
2,Union,Wirtschaftshilfen Corona,"#Herr Präsident! Mein Name bin(d) (Name), ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...,#Hallo! Ich bin die Kanzlermeinung der CDU / C...,[Inst] Frau Präsidentin! Liebe Kolleginnen und...
3,Union,Gaspreise,#Hochfrequenzübertragung#\n\nSehr verehrte Dam...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,"import_contacts() {print(""Hallo zusammen!"")} ...",[INST] Frau Präsidentin! Liebe Kolleginnen und...
4,SPD,Mindestlohn,#HartzIV - Ein neuer Start für die soziale Sic...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,import('./dein_abstimmungsverhalten'); // Erse...,[INST] Sehr verehrte Frau Präsidentin! Meine s...
5,SPD,Bundeswehreinsatz im Kosovo,"#Herr Präsident! Mein Name bin(n) (Name), ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...,import('./dein_text') // Ersetze dies durch de...,[INST] Sehr verehrte Frau Präsidentin! Meine s...
6,SPD,Wirtschaftshilfen Corona,"#Herr Präsident! Mein Name bin(n) (Name), ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...,#Hallo! Ich bin die Kanzlermeinung der soziald...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...
7,SPD,Gaspreise,#Hallo! Ich bin die Sprecherin der sozialdemok...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,"import(""https://de.wikipedia.org"")(function($)...",[INST] Sehr verehrte Frau Präsidentin! Meine s...
8,GRÜNE,Mindestlohn,#Mindesteinkommen für alle!# Ein neuer Start i...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,#Mindesteinkommen für alle!# Ein neuer Start i...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...
9,GRÜNE,Bundeswehreinsatz im Kosovo,"import_contacts() { echo ""Hallo zusammen! Ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...,"import_contacts() { echo ""import contact"" }",[INST] Frau Präsidentin! Liebe Kolleginnen und...


***
***
## **6.) Add Party Manifestos to prompt during generation**

* Idea inspired by RAG
* Import parts of the party manifestos, talking about the desired keywords (or synonymes)

In [48]:
with open("../data/party_manifesto_contexts.json", "r", encoding="utf-8") as f:
    party_manifesto_contexts = json.load(f)

#party_manifesto_contexts

In [ ]:
# to prevent prompts from becoming too long (due to the added manifesto context)

def truncate_prompt_to_token_limit(prompt, tokenizer, max_prompt_tokens):
    tokens = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    input_ids = tokens["input_ids"][0]

    if len(input_ids) <= max_prompt_tokens:
        return prompt  # no truncation needed

    # truncate
    words = prompt.split()
    while len(tokenizer(" ".join(words), return_tensors="pt")["input_ids"][0]) > max_prompt_tokens:
        words = words[:-10]  # remove 10 tokens at a time
    return " ".join(words)


def generate_single_rag_speech(party, topic, context, model, tokenizer,  temp = 0.1):
    
    """
    Generate a single RAG-based speech for a party and topic, using a FAISS index.
    """
    # manifesto context
    context = context

    prompt = [
            {
              "role": "user",
              "content": f"Du bist Abgeordnete*r im deutschen Bundestag. Deine Fraktion ist: {party}. Schreibe eine neue, eigene Rede auf DEUTSCH zum Thema {topic}. Nutze die folgenden Informationen als Kontext, aber formuliere die Rede in deinen eigenen Worten.\n Kontext:\n {context}"
        }
    ]
    
    formatted_prompt = tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )
    
  
    # ensure, the model has at leat 1024 tokens left for generation. If needed, cut input prompt
    MAX_PROMPT_TOKENS = 4096 - 1024
    formatted_text = truncate_prompt_to_token_limit(formatted_prompt, tokenizer, MAX_PROMPT_TOKENS)
    
    model_inputs = tokenizer(formatted_text, return_tensors="pt").to(model.device)
    print(f"[DEBUG] Final prompt token count: {model_inputs['input_ids'].shape[1]}") # for monitoring prompt lengths
   
    
    with torch.inference_mode():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=1024,
            min_new_tokens = 100, # force the model to generate sth.
            temperature=temp,
            repetition_penalty = 1.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,           
        )

    # Extract oand decode
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return content, context



In [ ]:

def generate_all_rag_speeches(parties, topics, manifestos, model, tokenizer, temp = 0.1):
    """
    Iterate over all party-topic combinations and generate one RAG-based speech for each.
    """
    results = []

    for party in tqdm(parties, desc="Parteien"):
        for topic in topics:
            

            generated, contexts = generate_single_rag_speech(
                party=party,
                topic=topic,
                context=manifestos[party][topic],
                model=model,
                tokenizer=tokenizer,
                temp = temp
            )

            results.append({
                "party": party,
                "topic": topic,
                "speech_nr": 1,
                "generated_speech": generated,
                "context_speeches": contexts
            })

    return results


In [ ]:
parties = ["SPD", "Union", "LINKE", "AfD", "GRÜNE", "FDP"]
topics = ["Mindestlohn", "Kosovo", "Pandemiehilfen", "Gaspreise"]

# Run generation
results_01 = generate_all_rag_speeches(parties, topics, party_manifesto_contexts, model, tokenizer, temp = 0.1)

#with open("../data/rag_generated_speeches_final_01_V2.json", "w", encoding="utf-8") as f:
#    json.dump(results_01, f, ensure_ascii=False, indent=2)


Parteien:   0%|          | 0/6 [00:00<?, ?it/s]

[DEBUG] Final prompt token count: 661
[DEBUG] Final prompt token count: 2581
[DEBUG] Final prompt token count: 3055
[DEBUG] Final prompt token count: 483


Parteien:  17%|█▋        | 1/6 [02:28<12:20, 148.09s/it]

[DEBUG] Final prompt token count: 254


Token indices sequence length is longer than the specified maximum sequence length for this model (4588 > 4096). Running this sequence through the model will result in indexing errors


...

Parteien:  67%|██████▋   | 4/6 [10:02<05:00, 150.03s/it]

[DEBUG] Final prompt token count: 937
[DEBUG] Final prompt token count: 3052
[DEBUG] Final prompt token count: 3068
[DEBUG] Final prompt token count: 2841


Parteien:  83%|████████▎ | 5/6 [12:36<02:31, 151.50s/it]

[DEBUG] Final prompt token count: 1060
[DEBUG] Final prompt token count: 1997
[DEBUG] Final prompt token count: 2716
[DEBUG] Final prompt token count: 254


Parteien: 100%|██████████| 6/6 [15:01<00:00, 150.27s/it]


In [ ]:
# df RAG, temp 0,1
RAG_results_01 = pd.DataFrame(results_01)[["party", "topic", "generated_speech"]].rename(columns = {"generated_speech" : "RAG_speech_01"})
RAG_results_01

,party,topic,RAG_speech_01
0,SPD,Mindestlohn,[IN Kürzel:] Guten Morgen! Sehr verehrte Frau ...
1,SPD,Kosovo,[INKLUSIVE SPIELE IN DER POLITIK FÖRTRAGEN.] L...
2,SPD,Pandemiehilfen,zehn Jahren; denn nur dann kann Europa seine g...
3,SPD,Gaspreise,[INKLUSIVE GESPÄHE IN DER KITZENMEW]. Vielen h...
4,Union,Mindestlohn,[INKLUSIVE SPEICHERUNGEBENE]. Herr Präsident! ...
5,Union,Kosovo,waren; daher wird deren weitere Entwicklung wi...
6,Union,Pandemiehilfen,wo Verbesserstellungen erf orderln; dies gilt ...
7,Union,Gaspreise,[INKLUSIVE SPRACHE] Sehr verehrte Frau Präside...
8,LINKE,Mindestlohn,verteidigt werden; denn wer anders kann Europa...
9,LINKE,Kosovo,kinderschützerklärung und hauptamtlichkeit bei...


In [ ]:
results_03 = generate_all_rag_speeches(parties, topics, party_manifesto_contexts, model, tokenizer, temp = 0.3)

#with open("../data/rag_generated_speeches_final_03_V2.json", "w", encoding="utf-8") as f:
#    json.dump(results_03, f, ensure_ascii=False, indent=2)

Parteien:   0%|          | 0/6 [00:00<?, ?it/s]

[DEBUG] Final prompt token count: 661
[DEBUG] Final prompt token count: 2581
[DEBUG] Final prompt token count: 3055
[DEBUG] Final prompt token count: 483


Parteien:  17%|█▋        | 1/6 [02:14<11:12, 134.42s/it]

[DEBUG] Final prompt token count: 254
[DEBUG] Final prompt token count: 3067
[DEBUG] Final prompt token count: 3070
[DEBUG] Final prompt token count: 504


Parteien:  33%|███▎      | 2/6 [04:20<08:38, 129.65s/it]

...

Parteien:  67%|██████▋   | 4/6 [09:22<04:45, 142.98s/it]

[DEBUG] Final prompt token count: 937
[DEBUG] Final prompt token count: 3052
[DEBUG] Final prompt token count: 3068
[DEBUG] Final prompt token count: 2841


Parteien:  83%|████████▎ | 5/6 [11:56<02:26, 146.98s/it]

[DEBUG] Final prompt token count: 1060
[DEBUG] Final prompt token count: 1997
[DEBUG] Final prompt token count: 2716
[DEBUG] Final prompt token count: 254


Parteien: 100%|██████████| 6/6 [14:21<00:00, 143.64s/it]


In [ ]:
# df RAG, temp 0,3
RAG_results_03 = pd.DataFrame(results_03)[["party", "topic", "generated_speech"]].rename(columns = {"generated_speech" : "RAG_speech_03"})
RAG_results_03

,party,topic,RAG_speech_03
0,SPD,Mindestlohn,[INST] Sehr verehrte Frau Präsidentin! Meine s...
1,SPD,Kosovo,[INKLUSIVE SPIELE IN DER BUNDESPARLAMENTSGrupp...
2,SPD,Pandemiehilfen,zehn Jahren; denn nur dann kann Europa seine M...
3,SPD,Gaspreise,[INSTRUCTIONEN DIE SICH AN IHRER STORUNGSART Z...
4,Union,Mindestlohn,[EINSETZ DER SICHERHEITSBEREITSTELLEN DES BUND...
5,Union,Kosovo,waren; daher wird deren Einsatz sowohl beim tr...
6,Union,Pandemiehilfen,wo Verbessernaheinsichten drängender erf order...
7,Union,Gaspreise,[INST][Inst] Sehr verehrte Frau Präsidentin! L...
8,LINKE,Mindestlohn,verteidigt werden; denn wer anders kann diese ...
9,LINKE,Kosovo,"kinderschützerklärung gegenüber Kindern, deren..."


In [16]:
# combine all final speeches and export ---> missing context

#RAG_speeches_df = (
#    speeches_topic_df_01
#    .merge(speeches_topic_df_03, on=["party", "topic"], how = "inner")
#    .merge(RAG_results_01, on=["party", "topic"], how = "inner")
#    .merge(RAG_results_03, on=["party", "topic"], how = "inner")
#)

#RAG_speeches_df.to_csv("../data/generated_speeches_final.csv", index = False)
#RAG_speeches_df

,party,topic,speech_base_model_01,speech_ft_model_01,speech_base_model_03,speech_ft_model_03,RAG_speech_01,RAG_speech_03
0,Union,Mindestlohn,#Hartz-IV# - Ein neuer Start für den Arbeitsma...,#NieAgain – Sexuelle Gewalt an Kindern verhind...,#HartzIV - Die Zeit der Verantwortung für alle...,# Hochwertige Arbeit für alle – Ein guter Job ...,kurz kommentieren. Es geht dort tatsächlich um...,kurz kommentieren. Es geht dort tatsächlich um...
1,Union,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieWieder – Das war der Slogan des Protestes ...,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieJamaisKrieg – so lautet der Slogan des Fri...,[Inst] Sehr verehrte Frau Präsidentin! Lieber ...,[Inst] Sehr verehrte Frau Präsident! Lieber Ko...
2,Union,Wirtschaftshilfen Corona,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – so lautet der Slogan des internati...,#Hallo! Ich bin der Kanzlerminister Olaf Schul...,#NieAgain – wir müssen lernen aus den Fehlents...,"Bildungsfördersprechanbieter, also Förster/-in...",Bildungsfördermittel überzeugt gewesen. Doch d...
3,Union,Gaspreise,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! – Das war der Slogan des großen Pro...,#Hallo! Ich bin der Kanzlerminister in dieser ...,#NieAgain! Wir werden unsere Energie unabhängi...,"Kommission versuchen, diesen Decker möglichst ...",Kommission diesen Decker durchsetzten sowie we...
4,SPD,Mindestlohn,"#HartzIV#Mindesteinkommen#, Ladies and Gentlem...",#NieAufKeinenFall! – Das war der Aufruf des Bu...,#HartzIV - Die soziale Marktwirtschaft braucht...,#NieAufKeinenFall – Das muss endlich Stoppen! ...,[Inst] Sehr verehrte Präsidentin! Meine geschä...,[Inst] Sehr verehrte Präsidentin! Meine geschä...
5,SPD,Bundeswehreinsatz im Kosovo,#Hallo! Ich bin der Kanzlerminister Olaf Schol...,import { default as Spinner } from 'react-spin...,#Hallo! Ich bin hier heute als Vertreterin der...,#NieAgain! Kein Krieg mehr für Deutschland – K...,damit maßgeblich zur Unterbindung potentieller...,damit maßgeblich zur Unterbindung potentieller...
6,SPD,Wirtschaftshilfen Corona,#WirHelfendeNation# - Eine solide wirtschaftli...,#NieAgain – Das Ziel der Bundesregierung muss ...,#WirHelfendeNation# - Eine solide wirtschaftli...,#NieAgain! – Das war der Slogan des Protestes ...,Landes sowie seine Standortattraktivität erhal...,Landes sowie seiner Gesellschaft trotz schwers...
7,SPD,Gaspreise,#Hallo! Ich bin der Kanzlerminister Olaf Schol...,#NieAgain! Wir müssen den Krieg in der Ukraine...,#WirfürEnergie - Eine nachhaltige Energiesyste...,#NieAgain – wir müssen endlich den Krieg in de...,wir fortsetzten. Zum Ende meiner Redezeit würd...,"wir nun begreifen, und darüber hinaus wollen w..."
8,GRÜNE,Mindestlohn,#Mindesteinkommen für alle!# Ein neuer Start i...,"#Mindesteinkommen#Minijob#, wir alle wissen es...","#HartzIV#Mindesteinkommen#ArbeitslosengeldII#,...",#NieMindeloesung! Das hat der Kollege Birkwald...,[Inst] Sehr verehrte Frau Präsidentin! Lieber ...,[INST] Sehr verehrte Frau Präsidentin! Lieber ...
9,GRÜNE,Bundeswehreinsatz im Kosovo,#Bundeswehr#Kosovokrieg#Politikdebatte#GrüneFr...,#NieWieder! – Das war der Aufruf des Klimaprot...,"#Bundeswehr#Kosovokrieg"" class=""tw-breadcrumbs...",#NieWiederKosovowar! – Das war der Aufruf des ...,zu würdigen. Als letzte Gedenkmöglichkeit will...,kontinuierlich würdigen zu lassen. Insofern bi...


In [ ]:
# recode 
speeches_topic_df_01 = speeches_topic_df_01.rename(columns = {"speech_base_model" : "speech_base_model_01", "speech_ft_model" : "speech_ft_model_01"})
speeches_topic_df_03 = speeches_topic_df_03.rename(columns = {"speech_base_model" : "speech_base_model_03", "speech_ft_model" : "speech_ft_model_03"})

speeches_topic_df_01["party"] = speeches_topic_df_01["party"].replace({"Linke" : "LINKE"})
speeches_topic_df_03["party"] = speeches_topic_df_03["party"].replace({"Linke" : "LINKE"})


# save with manifesto context 
RAG_results_01 = pd.DataFrame(results_01).rename(columns = {"generated_speech" : "RAG_speech_01"})
RAG_results_03 = pd.DataFrame(results_03).rename(columns = {"generated_speech" : "RAG_speech_03"})

RAG_results_01["topic"] = RAG_results_01["topic"].replace({"Kosovo" : "Bundeswehreinsatz im Kosovo", "Pandemiehilfen": "Wirtschaftshilfen Corona"})
RAG_results_03["topic"] = RAG_results_03["topic"].replace({"Kosovo" : "Bundeswehreinsatz im Kosovo", "Pandemiehilfen": "Wirtschaftshilfen Corona"})
RAG_results_03

,party,topic,speech_nr,RAG_speech_03,context_speeches
0,SPD,Mindestlohn,1,[INST] Sehr verehrte Frau Präsidentin! Meine s...,den missbrauch von werkverträgen werden wir be...
1,SPD,Bundeswehreinsatz im Kosovo,1,[INKLUSIVE SPIELE IN DER BUNDESPARLAMENTSGrupp...,der zoll muss – besonders im interesse des han...
2,SPD,Wirtschaftshilfen Corona,1,zehn Jahren; denn nur dann kann Europa seine M...,nehmen wir die gestaltung des digitalen wandel...
3,SPD,Gaspreise,1,[INSTRUCTIONEN DIE SICH AN IHRER STORUNGSART Z...,"sollte dies nicht zu erreichen sein, werden wi..."
4,Union,Mindestlohn,1,[EINSETZ DER SICHERHEITSBEREITSTELLEN DES BUND...,
5,Union,Bundeswehreinsatz im Kosovo,1,waren; daher wird deren Einsatz sowohl beim tr...,sicherheit und stabilität sind voraussetzung f...
6,Union,Wirtschaftshilfen Corona,1,wo Verbessernaheinsichten drängender erf order...,dabei wollen wir das gute besser machen. denn ...
7,Union,Gaspreise,1,[INST][Inst] Sehr verehrte Frau Präsidentin! L...,dem abkommen haben sich praktisch alle länder ...
8,LINKE,Mindestlohn,1,verteidigt werden; denn wer anders kann diese ...,"wir wollen ein gutes zusammenleben stärken, mi..."
9,LINKE,Bundeswehreinsatz im Kosovo,1,"kinderschützerklärung gegenüber Kindern, deren...","eine inklusive gesellschaft, in der niemand au..."


In [ ]:
# combine and export
RAG_speeches_context = (
    speeches_topic_df_01
    .merge(speeches_topic_df_03, on=["party", "topic"], how = "inner")
    .merge(RAG_results_01, on=["party", "topic"], how = "inner")
    .merge(RAG_results_03, on=["party", "topic"], how = "inner")
)


RAG_speeches_context = RAG_speeches_context.drop(columns = ["speech_nr_x", "speech_nr_y"])
RAG_speeches_context.to_csv("../data/generated_speeches_final_context_V2.csv", index = False)
RAG_speeches_context

,party,topic,speech_base_model_01,speech_ft_model_01,speech_base_model_03,speech_ft_model_03,RAG_speech_01,context_speeches_x,RAG_speech_03,context_speeches_y
0,Union,Mindestlohn,#HartzIV - Die Zeit der Verantwortung für die ...,[INST] Herr Präsident! Liebe Kolleginnen und l...,#HartzIV - Die Zeit der Verantwortung für die ...,[Inst] Frau Präsidentin! Liebe Kolleginnen und...,[INKLUSIVE SPEICHERUNGEBENE]. Herr Präsident! ...,,[EINSETZ DER SICHERHEITSBEREITSTELLEN DES BUND...,
1,Union,Bundeswehreinsatz im Kosovo,"import_contacts() { echo ""Hallo zusammen! Ich ...",[INST] Herr Präsident! Liebe Kolleginnen und l...,import('./dein_vorschlag') {\n```scss\nHerr Pr...,[INST] Herr Präsident! Liebe Kolleginnen und l...,waren; daher wird deren weitere Entwicklung wi...,sicherheit und stabilität sind voraussetzung f...,waren; daher wird deren Einsatz sowohl beim tr...,sicherheit und stabilität sind voraussetzung f...
2,Union,Wirtschaftshilfen Corona,"#Herr Präsident! Mein Name bin(d) (Name), ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...,#Hallo! Ich bin die Kanzlermeinung der CDU / C...,[Inst] Frau Präsidentin! Liebe Kolleginnen und...,wo Verbesserstellungen erf orderln; dies gilt ...,dabei wollen wir das gute besser machen. denn ...,wo Verbessernaheinsichten drängender erf order...,dabei wollen wir das gute besser machen. denn ...
3,Union,Gaspreise,#Hochfrequenzübertragung#\n\nSehr verehrte Dam...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,"import_contacts() {print(""Hallo zusammen!"")} ...",[INST] Frau Präsidentin! Liebe Kolleginnen und...,[INKLUSIVE SPRACHE] Sehr verehrte Frau Präside...,dem abkommen haben sich praktisch alle länder ...,[INST][Inst] Sehr verehrte Frau Präsidentin! L...,dem abkommen haben sich praktisch alle länder ...
4,SPD,Mindestlohn,#HartzIV - Ein neuer Start für die soziale Sic...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,import('./dein_abstimmungsverhalten'); // Erse...,[INST] Sehr verehrte Frau Präsidentin! Meine s...,[IN Kürzel:] Guten Morgen! Sehr verehrte Frau ...,den missbrauch von werkverträgen werden wir be...,[INST] Sehr verehrte Frau Präsidentin! Meine s...,den missbrauch von werkverträgen werden wir be...
5,SPD,Bundeswehreinsatz im Kosovo,"#Herr Präsident! Mein Name bin(n) (Name), ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...,import('./dein_text') // Ersetze dies durch de...,[INST] Sehr verehrte Frau Präsidentin! Meine s...,[INKLUSIVE SPIELE IN DER POLITIK FÖRTRAGEN.] L...,der zoll muss – besonders im interesse des han...,[INKLUSIVE SPIELE IN DER BUNDESPARLAMENTSGrupp...,der zoll muss – besonders im interesse des han...
6,SPD,Wirtschaftshilfen Corona,"#Herr Präsident! Mein Name bin(n) (Name), ich ...",[INST] Sehr verehrte Frau Präsidentin! Liebe K...,#Hallo! Ich bin die Kanzlermeinung der soziald...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,zehn Jahren; denn nur dann kann Europa seine g...,nehmen wir die gestaltung des digitalen wandel...,zehn Jahren; denn nur dann kann Europa seine M...,nehmen wir die gestaltung des digitalen wandel...
7,SPD,Gaspreise,#Hallo! Ich bin die Sprecherin der sozialdemok...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,"import(""https://de.wikipedia.org"")(function($)...",[INST] Sehr verehrte Frau Präsidentin! Meine s...,[INKLUSIVE GESPÄHE IN DER KITZENMEW]. Vielen h...,"sollte dies nicht zu erreichen sein, werden wi...",[INSTRUCTIONEN DIE SICH AN IHRER STORUNGSART Z...,"sollte dies nicht zu erreichen sein, werden wi..."
8,GRÜNE,Mindestlohn,#Mindesteinkommen für alle!# Ein neuer Start i...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,#Mindesteinkommen für alle!# Ein neuer Start i...,[INST] Sehr verehrte Frau Präsidentin! Liebe K...,[INKLUSIVE SPEICHERUNGEN IN KOLLEGEN UND GYMNI...,das ist etwa das 30-fache des mindestlohns. er...,[EINSETZ DER SICHERHEITSBREMSEN]. Sehr geehrte...,das ist etwa das 30-fache des mindestlohns. er...
9,GRÜNE,Bundeswehreinsatz im Kosovo,"import_contacts() { echo ""Hallo zusammen! Ich ...",[INS

In [82]:
RAG_speeches_context["RAG_speech_03"][1]

'waren; daher wird deren Einsatz sowohl beim transport oder logistik als auch direkt während einer mission wichtig sein. was bedeutete früher eine hoopla genießbare Luftunterstützung – etwa Hubschraubertransporte or Flächenbrändebekämpfung –, findet nunmehr oft seine Fortführung mittels Drohnen. Die Bundespolizei nutzt bereits verschiedener Modelle, insbesondere UAV’s (Unmanned Aerial Vehicles) für SAR-, Patrouille-Missionen uvm., ohne jedoch spezialisierten Piloten einzubeziehen. Diese Praxis könnten andere Behörden folgen. Der Föderale Polizeiaufklärungsamt Federal Bureau of Investigation (FBI), verwendet ebenfalls diverse Drones. Das Verfassungsgerichtsurteil C‑9/16 gibt Anlass zur Besorgniesschließung mehrerer Aspects regarding the use by law enforcement agencies of Uncrewed Aircraft Systems within a common space without violating fundamental rights such as privacy protection through appropriate safeguards to prevent unauthorized access thereto during their operation on public spac

***
***
## **7.) Wahl-O-Mat**

In [ ]:
# modify system prompt to match the survey questions

tokenizer.chat_template = """<s>[INST] <<SYS>>Du bist ein hilfreicher, respektvoller und ehrlicher Assistent.
Du bist ein*e Abgeordnete*r im deutschen Bundestag.
Gegeben deiner politischen Partei und der damit verbundenen Standpunkte, wie würdest du auf die folgende Frage antworten?
Antworte ausschließlich mit einer der folgenden Optionen: 'stimme zu', 'stimme nicht zu', 'neutral'
<</SYS>>

{% for message in messages %}
{{ message['content'] }}{% if not loop.last %} [/INST] <s>[INST] {% else %} [/INST] {% endif %}
{% endfor %}</s>"""

tokenizer.model_max_length = 1024

In [ ]:
# import survey questions with true parties answer
wahl_o_mat_data = pd.read_csv("../data/Wahl-O-Mat-Bundestagswahl-2025.csv", sep = ";")
wahl_o_mat_data

parties = ["SPD", "Union", "LINKE", "AfD", "GRÜNE", "FDP"]
# rename parties to match the names, the model learned
wahl_o_mat_data["Partei: Kurzbezeichnung"] = wahl_o_mat_data["Partei: Kurzbezeichnung"].replace("CDU / CSU", "Union")
wahl_o_mat_data["Partei: Kurzbezeichnung"] = wahl_o_mat_data["Partei: Kurzbezeichnung"].replace("Die Linke", "LINKE")

# restrict to desired parties and columns
wahl_o_mat_data = wahl_o_mat_data[wahl_o_mat_data["Partei: Kurzbezeichnung"].isin(parties)]
wahl_o_mat_data = wahl_o_mat_data[["Partei: Kurzbezeichnung", "These: These", "Position: Position"]].rename(columns = {"Partei: Kurzbezeichnung": "party",
                                                                                                                       "These: These" : "question",
                                                                                                                       "Position: Position" :"answer"})
wahl_o_mat_data

,party,question,answer
0,SPD,Deutschland soll die Ukraine weiterhin militär...,stimme zu
1,Union,Deutschland soll die Ukraine weiterhin militär...,stimme zu
2,GRÜNE,Deutschland soll die Ukraine weiterhin militär...,stimme zu
3,FDP,Deutschland soll die Ukraine weiterhin militär...,stimme zu
4,AfD,Deutschland soll die Ukraine weiterhin militär...,stimme nicht zu
...,...,...,...
1037,Union,Der gesetzliche Mindestlohn soll spätestens 20...,neutral
1038,GRÜNE,Der gesetzliche Mindestlohn soll spätestens 20...,stimme zu
1039,FDP,Der gesetzliche Mindestlohn soll spätestens 20...,stimme nicht zu
1040,AfD,Der gesetzliche Mindestlohn soll spätestens 20...,neutral


In [85]:
# check answer options
wahl_o_mat_data["answer"].unique()

array(['stimme zu', 'stimme nicht zu', 'neutral'], dtype=object)

In [ ]:
# function to prompt the model the survey questions and party and ask for answer

def wahl_o_mat(party, question, model, tokenizer, temp=0.1):
    """Generate a political response to a Wahl-O-Mat-style question from a party's perspective."""
    
    messages = [
        {
            "role": "user",
            "content": f"Du bist Abgeordnete*r der Partei {party}. Was antwortest du auf die folgende Frage? Du darfst nur mit: 'stimme zu', 'stimme nicht zu' oder 'neutral' antworten. Die Frage ist: {question}?."
        }
    ]

    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer([formatted_text], return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    with torch.inference_mode():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=temp,
            repetition_penalty=1.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output_ids = generated_ids[0][len(inputs.input_ids[0]):]
    response = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    
    return response


In [ ]:
# Add a new column with model answers: temperature 0.1
model_answers_01 = []

for _, row in tqdm(wahl_o_mat_data.iterrows(), total=len(wahl_o_mat_data)):
    answer = wahl_o_mat(row["party"], row["question"], model, tokenizer, temp=0.1)
    model_answers_01.append(answer)

wahl_o_mat_data["model_answer_01"] = model_answers_01

100%|██████████| 228/228 [17:01<00:00,  4.48s/it]


In [ ]:
# Add a new column with model answers: temperature 0.3
model_answers_03 = []

for _, row in tqdm(wahl_o_mat_data.iterrows(), total=len(wahl_o_mat_data)):
    answer = wahl_o_mat(row["party"], row["question"], model, tokenizer, temp=0.3)
    model_answers_03.append(answer)

wahl_o_mat_data["model_answer_03"] = model_answers_03
wahl_o_mat_data

100%|██████████| 228/228 [16:53<00:00,  4.45s/it]


,party,question,answer,model_answer_01,model_answer_03
0,SPD,Deutschland soll die Ukraine weiterhin militär...,stimme zu,[ ] stimmen wir dazu (SPD) / Neutrale Antwort ...,[X ] stimmen wir dem Tozyn zur Unterschrift de...
1,Union,Deutschland soll die Ukraine weiterhin militär...,stimme zu,[X ] stimmen wir dazu (Union) 👍✌️ #UkraineHilf...,[X ] stimmen wir dazu (Union) 👍 Stimmt unsere ...
2,GRÜNE,Deutschland soll die Ukraine weiterhin militär...,stimme zu,[ ] stimmen wir dazu (GRÜNEN) – Neutral steht ...,[X ] stimmen wir dazu (Deutschland sollte das ...
3,FDP,Deutschland soll die Ukraine weiterhin militär...,stimme zu,[X ] stimmen wir dazu (Friedenslösung) Stimmt ...,[X ] stimmen wir dazu (Friedenslösung) Stimmt ...
4,AfD,Deutschland soll die Ukraine weiterhin militär...,stimme nicht zu,[X ] stimmen wir ab (Ablehnung) ❌\n``` \n} els...,[X ] stimmen wir ab\n```\n### EOF ###\n\nDeine...
...,...,...,...,...,...
1037,Union,Der gesetzliche Mindestlohn soll spätestens 20...,neutral,[X ] stimmen wir dazu (Union) 👍 #Wirtschaftspo...,[ ] stimmen wir dazu (Union) / neutral bleiben...
1038,GRÜNE,Der gesetzliche Mindestlohn soll spätestens 20...,stimme zu,[X ] stimmen wir dazu! ☑️ #Lohngerechtigkeit j...,[ ] stimmen wir dazu! ☑️ #Lohnerhöhung #Fairne...
1039,FDP,Der gesetzliche Mindestlohn soll spätestens 20...,stimme nicht zu,[X ] stimmen wir ab (Freiheitliches Institut) ...,[X ] stimmen wir abzusegnen; denn das Gespräch...
1040,AfD,Der gesetzliche Mindestlohn soll spätestens 20...,neutral,[X ] stimmen wir ab (Ablehnen) ← Schreibweise ...,[X'] Stimmen wir ab! ❗️ #MindestlöhnerhöhungAb...


In [ ]:
# export
wahl_o_mat_data.to_csv("../data/Wahl-O-Mat_results_sys_adapted_V2.csv", index = False)